In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [40]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(4, 8, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Conv2d(8, 24, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Linear(24 * 5 * 5, 512),
    #torch.nn.ReLU(),
    #torch.nn.Linear(512, 256),
    #torch.nn.ReLU(),
    #torch.nn.Linear(256, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [4]:
model = torch.load('./models/1epochAdam.pt')
model.train()
model.to(device)
if cuda_status:
    model = model.cuda()

### Visualize the batch of sequences

In [7]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 3

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


tensor(307.8059, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(340.8285, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(379.1136, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(360.9464, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(215.2606, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(136.1281, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(82.3836, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(85.9909, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(68.1346, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(52.9967, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(47.1554, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(37.9628, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(35.8901, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.9262, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.3300, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.6468, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.3073, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.5702, device='c

tensor(46.1739, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(53.7488, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.5690, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(52.6609, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(67.1655, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(58.5320, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(41.2759, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(33.3216, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.2841, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(24.6502, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.4095, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(27.3572, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.7088, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.0263, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.4972, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.2673, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.7607, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.0850, device='cuda:0'

tensor(30.7582, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(33.1994, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(39.9872, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(45.7979, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(52.6128, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(55.8503, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.1340, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.2786, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.6851, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.5539, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.7245, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.4135, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.4784, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.0188, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.3870, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.6501, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.3957, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.6719, device='cuda:0',

tensor(34.9047, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(38.2809, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(44.8045, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(46.9546, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(66.7946, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(54.4398, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.4605, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.6942, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(66.5517, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.6548, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.7289, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(64.1367, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(82.9498, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(47.0611, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(46.6824, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.5993, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.3006, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.0191, device='cuda:0'

tensor(32.2320, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.4297, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.8752, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.2740, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.7852, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(27.8944, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(40.0113, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(43.4649, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(58.9147, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(60.0446, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.7995, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(55.3171, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(68.8770, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(33.1606, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.9425, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.0764, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.8281, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.4213, device='cuda:0'

...

tensor(14.3461, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.5099, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.8645, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.5338, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.9392, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.9378, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.0845, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.6361, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.2181, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.0365, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.4986, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.7409, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.7859, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.5630, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.9243, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.7376, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.4521, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.0548, device='cuda:0', 

tensor(23.9600, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.4080, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.8416, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.5164, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.4265, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.5414, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.2596, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.8875, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.2537, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.9452, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.6037, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.0050, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.2050, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.8455, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.9279, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.2267, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.4273, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.4261, device='cuda:0'

tensor(8.7856, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.5076, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.6049, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.5039, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.8026, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.3090, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.1068, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.4048, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.6872, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.7157, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.2150, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.6490, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(6.6877, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.9851, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.7540, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.0082, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.0121, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.0377, device='cuda:0', grad_fn=<SqrtBa

tensor(3.6676, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.2291, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.2497, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.9126, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.0622, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.3637, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.6516, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(6.2996, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.9374, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.6245, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.9281, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.2939, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.8841, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.9463, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.2479, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.0974, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.8133, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.0612, device='cuda:0', grad_fn=<SqrtBac

tensor(2.5827, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.7097, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.8323, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.8891, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.6137, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.4241, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.5637, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.3982, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.6085, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.2070, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.8340, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.6173, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.7527, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.3498, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.8887, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.9959, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.9288, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.9025, device='cuda:0', grad_fn=<SqrtBac

In [6]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 3
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        #x = torch.flatten(inp, start_dim=2)

        x = inp.permute(0,3,1,2).float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        print(y_pred.shape)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


RuntimeError: size mismatch, m1: [15360 x 19], m2: [76 x 98] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:273

In [7]:
torch.save(model, './models/2epochAdamOffOf1.pt')

In [4]:
model = torch.load('./models/6epochmodel.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [8]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                


         ID                  v1                  v2                  v3  \
0     10002   1655.964599609375     453.40087890625     1660.8994140625   
1     10015   734.0613403320312       1225.95703125     717.70654296875   
2     10019   576.8329467773438      1244.896484375    576.530517578125   
3     10028  1623.5098876953125   437.8617248535156    1630.07958984375   
4      1003   2106.266845703125   685.2469482421875    2107.18896484375   
...     ...                 ...                 ...                 ...   
3195   9897   259.7702331542969      803.6533203125  259.81292724609375   
3196     99   593.4232788085938   1149.840576171875         585.8828125   
3197   9905  1716.0064697265625   505.3529052734375     1718.5498046875   
3198   9910   577.2566528320312    1288.65087890625    575.027099609375   
3199   9918   593.0121459960938  1159.1773681640625   583.0406494140625   

                      v4                  v5                  v6  \
0      471.0829772949219  1657